In [3]:
# !pip install transformers

In [1]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertModel.from_pretrained("bert-large-uncased")


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [2]:
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [3]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [4]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # Will be True if GPU is accessible


2.7.1+cu118
True


In [8]:
text1 = "I love  programming"
text2 = "i enjoy coding"

token1 = tokenizer(text1,return_tensors='pt')
token2 = tokenizer(text2,return_tensors='pt')

token1,token2

({'input_ids': tensor([[ 101, 1045, 2293, 4730,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[  101,  1045,  5959, 16861,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])})

In [10]:
embedding1 = model(**token1)['last_hidden_state']
embedding2 = model(**token2)['last_hidden_state']

embedding1,embedding2

(tensor([[[-0.0853, -0.5765, -0.6334,  ..., -0.8695, -0.6149,  0.1350],
          [-0.0676, -0.5738, -0.2257,  ..., -0.2472, -0.1451, -0.2964],
          [-0.3468, -0.2305, -0.0659,  ..., -0.3865,  0.0885,  0.0650],
          [ 0.1064, -0.1514, -0.1836,  ...,  0.1019, -0.7688,  0.1537],
          [-0.7420, -0.2230, -0.1059,  ..., -0.7624, -0.5671, -0.1679]]],
        grad_fn=<NativeLayerNormBackward0>),
 tensor([[[-0.3767, -0.9047, -0.5043,  ..., -0.9634, -0.5423,  0.1136],
          [-0.0535, -0.5905, -0.4027,  ..., -0.1526, -0.1549, -0.2906],
          [-0.2147, -0.1513, -0.2134,  ..., -0.3337,  0.1453, -0.3278],
          [ 0.1037,  0.0682, -0.3261,  ...,  0.0596, -0.7203, -0.0162],
          [-0.9850,  0.2231,  0.0861,  ..., -0.6314, -0.7672, -0.4168]]],
        grad_fn=<NativeLayerNormBackward0>))

In [11]:
print("Embedding 1 shape:", embedding1.shape)
print("Embedding 2 shape:", embedding2.shape)

Embedding 1 shape: torch.Size([1, 5, 1024])
Embedding 2 shape: torch.Size([1, 5, 1024])


In [16]:
import numpy as np 

mean_embedding1 = np.mean(embedding1.detach().numpy(),axis=1)
mean_embedding2 = np.mean(embedding2.detach().numpy(),axis=1)
# mean_embedding3 = np.mean(embedding3.detach().numpy(),axis=1)

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(mean_embedding1,mean_embedding2)

array([[0.9234957]], dtype=float32)

In [19]:
text1 = "I love  programming"
text2 = "i enjoy coding"
text3 = "I ate apple everyday"

token1 = tokenizer(text1,return_tensors='pt')
token2 = tokenizer(text2,return_tensors='pt')
token3 = tokenizer(text3,return_tensors='pt')

embedding1 = model(**token1)['last_hidden_state']
embedding2 = model(**token2)['last_hidden_state']
embedding3 = model(**token3)['last_hidden_state']


mean_embedding1 = np.mean(embedding1.detach().numpy(),axis=1)
mean_embedding2 = np.mean(embedding2.detach().numpy(),axis=1)
mean_embedding3 = np.mean(embedding3.detach().numpy(),axis=1)

cosine_similarity(mean_embedding1,mean_embedding2),cosine_similarity(mean_embedding1,mean_embedding3),cosine_similarity(mean_embedding3,mean_embedding2)



(array([[0.9234957]], dtype=float32),
 array([[0.73619556]], dtype=float32),
 array([[0.7747085]], dtype=float32))

In [21]:
text1 = "I love programming"
text2 = "i enjoy coding"
text3 = "I hat dogs"

token1 = tokenizer(text1, return_tensors='pt')
token2 = tokenizer(text2, return_tensors='pt')
token3 = tokenizer(text3, return_tensors='pt')

embedding1 = model(**token1)['last_hidden_state']
embedding2 = model(**token2)['last_hidden_state']
embedding3 = model(**token3)['last_hidden_state']

# Detach from graph and convert to numpy, then compute mean along axis 1 (sequence length)
# The output of mean will be (batch_size, hidden_size)
mean_embedding1 = np.mean(embedding1.detach().numpy(), axis=1)
mean_embedding2 = np.mean(embedding2.detach().numpy(), axis=1)
mean_embedding3 = np.mean(embedding3.detach().numpy(), axis=1)

# Ensure the embeddings are 2D arrays (even if batch_size is 1)
# The cosine_similarity function from sklearn expects 2D arrays
# (n_samples, n_features). If batch_size is 1, mean_embeddingX will be (1, hidden_size).
# If you don't use .reshape(1, -1), it might be (hidden_size,) which will cause an error.
mean_embedding1 = mean_embedding1.reshape(1, -1)
mean_embedding2 = mean_embedding2.reshape(1, -1)
mean_embedding3 = mean_embedding3.reshape(1, -1)

# Compute cosine similarities
sim_1_2 = cosine_similarity(mean_embedding1, mean_embedding2)
sim_1_3 = cosine_similarity(mean_embedding1, mean_embedding3)
sim_3_2 = cosine_similarity(mean_embedding3, mean_embedding2)

print(f"Similarity between '{text1}' and '{text2}': {sim_1_2[0][0]:.4f}")
print(f"Similarity between '{text1}' and '{text3}': {sim_1_3[0][0]:.4f}")
print(f"Similarity between '{text3}' and '{text2}': {sim_3_2[0][0]:.4f}")

Similarity between 'I love programming' and 'i enjoy coding': 0.9235
Similarity between 'I love programming' and 'I hat dogs': 0.7430
Similarity between 'I hat dogs' and 'i enjoy coding': 0.7368
